# Introduction

**Online advertising** is the main method used by search engine companies to generate revenue. Companies such as Google, Facebook, and Yahoo!, are all large providers of search advertising. Advertisers can bid on each search query (or "keyword") posted by the engine's users. In comparison with traditional advertising (TV, newspaper, etc.), online advertising has several advantages, explaining its popularity:
 - *Real-time execution:* Advertisers can bid in real-time to display their ads to the engine's users, without any further delay. Most parameters of the ads can be controlled in real-time.
 - *Data-driven:* Online advertising is an information-rich environment. Performance metrics and user-level attributes are made available to inform, analyse and optimize the bidding process.
 


**Google** is a popular search engine, and thus, it is a frequently used option for advertisers to place their ads. AdWords, the main advertising product of Google, was launched in 2000 to increase Google's revenue. Their opening pitch was "Have a credit card and 5 minutes? Get you ad on Google". Nowadays, AdWords allows advertisers to place bids for ganular search queries in order to display their ads on the search page. The original idea was invented by Bill Gross from Idealab. The similarities in the two systems led to some legal action, but the dispute was settled out of court, and AdWords continued unharmed. AdWords has been a very successful product for Google, which historically provided Google with over 90% of its annual revenue (2000-2014).

 **Optimization methods** create a substantial edge for online advertising platforms such as Google. At a high-level, by using prescriptive analytics, the online platforms can ensure that the right advertisement content is displayed to the right user at the right price. Every day, Google solves linear programs to decide on the allocation of billions of ads!

 **Your assignment:** In what follows, you will take the perspective of Google's AdWords platform:

1. Business schools in London want to advertise their master level programs on Google. Suppose that each user can see at most one ad.
- You have forecasts describing the traffic to the Google search engine. There is limited *number of user search queries* (or capacity) for each keyword. Each keyword also has a distinct *click-through rate*.
- The business schools input their daily *budgets*, and how much they *value* a click on each keyword. This is the revenue Google makes per click.
- Google Adwords uses linear programming to optimize the allocation of business school ads to the incoming search queries in order to maximize the total expected revenue.

# Q1. Data

We begin by importing the GUROBI package and the data.

In [1]:
# The following lines of code import the gurobi package
import gurobipy as gp
from gurobipy import GRB,quicksum

There are four potential keywords. For each keyword, there is a daily number of users that will make a search query on the keyword in question (i.e., capacity). For example, there are 10,000 user search queries for the "london" keyword. For simplicity, we will assume that Google can display at most one business school to each user.

In [2]:
# Next dictionary contains the capacities of each keyword
keywords = ['london','master_program', 'data_science']
capacities = {
    'london': 10000,
    'master_program': 2500,  
    'data_science': 2380
}

Google uses historical data to predict the click-through rate of each keyword. This is the probability of a click per user impression.

In [3]:
clickRatePerKeyword = {
    'london': 0.001,
    'master_program': 0.003,  
    'data_science': 0.003, 
}

There are two business schools, each of which has specified a total budget.

In [4]:
# Next dictionary contains the £ budget of each business school
bSchools = ['LBS','Imperial']
budgets = {
    'LBS': 50000,
    'Imperial': 25000,    
}

Each business school informs the Google ad exchange about how much it values each "click" per keyword. This is the revenue Google charges the business school for each click.

In [5]:
# Next dictionary contains the keywords specified by each business school
revenuePerKeyword = {
    ('LBS','london'): 2000,
    ('LBS','master_program'): 3200,  
    ('LBS','data_science'): 4000, 
    ('Imperial','london'):0,
    ('Imperial','master_program'): 500,  
    ('Imperial','data_science'): 3500,  
}

We now construct another dictionary that will be also very useful.

In [6]:
extra_dictionary = {
    (b,k): clickRatePerKeyword[k]*revenuePerKeyword[b,k] for b in bSchools for k in keywords
}

# About the Python syntax: 
# You can use "for loops" inside dictionaries, lists, etc.
# Python will consider all the elements of the loop to construct the dictionary/list
# You can even combine multiple loops, as in the example above.
# This is equivalent to the following code:
# 
# extra_dictionary = {}
# for b in bSchools:
#     for k in keywords:
#         extra_dictionary[b,k] = clickRatePerKeyword[k]*revenuePerKeyword[b,k]

### Q1.1. What does this dictionary represent? Explain the calculation and what it means.


This dictionary represents the revenue per impression for each keyword and business schools. The caculation multiplies the click through rate for each keyword by the revenue Google charges the business school for each click.As click through rate = clicks/impression, it means the revenue Google can generate for each impression they provide for each keyword specified by each business school.

# Q2.  Google Adwords' Allocation Linear Program

Recall that the LP formulation has three components:
- Decision variables: What variables does Google Adwords control?
- Constraints: What limitations does the platform face?
- Objective function: What is the performance metric? Are we *minimizing* or *maximizing*?

# Formulate Google Adwords' LP

Solution:
- Decision variables: *Google needs to decide how many users (from each keyword) are allocated to each business school ad.*
- Constraints: *There is a limited number of users for each keyword. Google has a limited budget for each business school.*
- Objective function: *The goal for Google is to maximize the expected revenue.*

In words, we obtain the following linear program:

$$ {\max} \quad \quad{\rm Total\ Expected\ Revenue} $$
$\quad\quad{\rm subject \ to}$
$$ \quad  {\rm LBS's\ Expected\ Cost} \leq {\rm Budget}_{LBS}$$
$$ \quad  {\rm Imperial's\ Expected\ Cost} \leq {\rm Budget}_{Imperial} $$
$$ \quad  {\rm Number\ of\ Users\ for\ Query\ "London"} \leq {\rm Capacity}_{lon}$$
$$ \quad  {\rm Number\ of\ Users\ for\ Query\ "Master\ Program"} \leq {\rm Capacity}_{mas}$$
$$ \quad  {\rm Number\ of\ Users\ for\ Query\ "Consulting\ Training"} \leq {\rm Capacity}_{con}$$
$$ \quad  {\rm Number\ of\ Users\ for\ Query\ "Data\ Science"}\leq {\rm Capacity}_{dat}$$


In mathematical form:

$$ \max_{x_{B,k} \geq 0} \quad\quad\quad  2.0\cdot x_{LBS,lon} +  9.6\cdot x_{LBS,mas} + 12.0\cdot x_{LBS,dat}+  1.5\cdot x_{Im,mas} + 10.5\cdot x_{Im,dat} $$
$ \quad \quad \text{subject to}$
$$ \quad  2.0\cdot x_{LBS,lon} +  9.6\cdot x_{LBS,mas} + 12.0\cdot x_{LBS,dat} \leq 50000$$
$$ \quad   1.5\cdot x_{Im,mas} + 10.5\cdot x_{Im,dat} \leq 25000 $$
$$ \quad  x_{LBS,lon} +  x_{Im,lon} \leq 10000$$
$$ \quad  x_{LBS,mas} +  x_{Im,mas} \leq 2500$$
$$ \quad  x_{LBS,dat} +  x_{Im,dat} \leq 2380$$

## Create a model

In what follows, we construct a linear program. *Fill and execute the cells below*

In [39]:
# We create a model object named "Adwords"
m = gp.Model("adwords")

## Step 1: Add decision variables

We create variables to decide on how many user queries of each keyword are served with ads from each business school.

In [40]:
# We create a dictionary of decision variables named "allocation"
# The first two arguments indicate that the allocation varies 
# by business school (first argument) and keyword (second argument)
allocation = m.addVars(bSchools,keywords, lb = 0, name="allocation")

In [41]:
allocation

{('LBS', 'london'): <gurobi.Var *Awaiting Model Update*>,
 ('LBS', 'master_program'): <gurobi.Var *Awaiting Model Update*>,
 ('LBS', 'data_science'): <gurobi.Var *Awaiting Model Update*>,
 ('Imperial', 'london'): <gurobi.Var *Awaiting Model Update*>,
 ('Imperial', 'master_program'): <gurobi.Var *Awaiting Model Update*>,
 ('Imperial', 'data_science'): <gurobi.Var *Awaiting Model Update*>}

## Step 2: Add constraints

### Add the constraints relative to the capacity of each keyword in the cell below. 
*Hint: Use the construct `m.addConstrs()` or `m.addConstr()`. To calculate a sum, you can use the construct `quicksum()`*

In [42]:
# Uncomment and complete the code below
capacity_constraints = m.addConstrs((quicksum(allocation[b,k] for b in bSchools) <= capacities[k] for k in keywords),'keyword_capacity')

#for k in keywords

### Now, add the constraints on the total budget of each business school  in the cell below.
*Hint: Use the construct `m.addConstr()`. To calculate a sum, you can use the construct `quicksum()`*

In [43]:
# Uncomment and complete the code below
budget_constraints = m.addConstrs((quicksum(extra_dictionary[b,k] * allocation[b,k] for k in keywords) <= budgets[b] for b in bSchools),'budget')

# for b in bSchools:
#

## Step 3: Specify the objective

We proceed by defining the objective of Google.

In [44]:
# The objective is to maximize the total expected revenue
m.setObjective(quicksum(allocation[b,k]*extra_dictionary[b,k] for b in bSchools for k in keywords), GRB.MAXIMIZE)

# Remark: This is equivalent to the following formulation:
# m.setObjective(allocation.prod(revenues), GRB.MINIMIZE)

## Solve the LP

Congratulations! You have formulated the Google Adwords LP. We will now solve it and printout the solution.

In [45]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nRevenue: %g' % m.objVal)
        print('\nAllocation of user queries:')
        allocationx = m.getAttr('x', allocation)
        for B in bSchools:
            for k in keywords:            
                print('%s %s %g ' % (B,k, allocationx[B,k]))
        print('\nRevenue per school per keyword:')
        allocationx = m.getAttr('x', allocation)
        for B in bSchools:
            for k in keywords:            
                print('%s %s %g ' % (B,k,allocationx[B,k]*extra_dictionary[B,k]))                
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 5 rows, 6 columns and 11 nonzeros
Model fingerprint: 0xa8cadc7d
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 4 rows, 5 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2560000e+04   7.050000e+02   0.000000e+00      0s
       1    6.9740000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.974000000e+04

Revenue: 69740

Allocation of user queries:
LBS london 10000 
LBS master_program 2500 
LBS data_science 500 
Imperial london 0 
Imperial master_program 0 
Imperial data_science 1880 

Revenue per school per keyword:
LBS london 20000 

### Q2.1. According to the LBS recruitment team, the keyword "data science"  usually attracts better candidates for the programs. Should LBS be concerned by the current allocation? How could LBS improve its allocation?
*Hint: Feel free to vary the inputs of the linear program to better understand its workings.*

Yes,because currently Imperial get more allocation for data_science keyword(1800) compared to LBS(500), which may attracts better candidates for the program. To improve its allocation, LBS can change how much it values each "click" per keyword. For example, LBS can lower the value for each "click" per keyword for keyword "london" and "master_program", and realatively increases this value for "data_science" to get more allocation in keyword "data_science".


# Q3. Should Google attract more traffic?

Google has the ability to attract more traffic to the search engine through various methods. Specifically, Google can increase the capacity of any given keyword. It  costs the same amount to attract more users searching for the keywords "london", "master_program" and "data_science". Google would like to understand for which keyword it is the most lucrative to slightly increase their capacity.

## Sensitivity analysis and shadow prices

As seen in class, once a linear program is formulated and solved, we can perform a sensitivity analysis. This analysis allows us to understand by how much the objective changes by slightly modifying the constraints. Specifically, the notion of shadow price quantifies how the optimal solution of a linear program varies relative to changes in the constant appearing on the righthand side of a given constraint. The shadow price of a constraint C is:

$$ {\rm shadowPrice(C)} = \frac{\Delta (\text{optimal value of the LP})}{ \Delta (\text{RHS of the constraint C})}$$

GUROBI automatically gives you the shadow price of each constraint. You may use the construct `constr.Pi` to get the shadow price of constraint `constr`.

### Q3.1: Using shadow prices, what advice can you give to Google? How would you prioritize increasing the keywords' capacity?

*Insert your answer here:*
Using shadow prices, Google can adjust the capacity of each keyword to increase ite revenue.I would prioritize increasing the keywords' capacity as 'data_science','master_program','london'.

In [52]:
# Insert your code here
for k in keywords:
    print(k,capacity_constraints[k].Pi)

london 1.75
master_program 8.4
data_science 10.5


### Q3.2: Which business school should Google encourage to increase its budget?

*Insert your answer here:*
According to the shadow prices, Google should encourage LBS to increase its budget. 

In [50]:
# Insert your code here
for b in bSchools:
    print(b,budget_constraints[b].Pi)

LBS 0.125
Imperial 0.0


# Q4. A more realistic click rate prediction

The notion that clicks grows linearly with the number of impressions is unrealistic. In reality, you would expect that, as you increase the number of impressions, it is more difficult generate additional clicks. This is due to marketing fatigue or saturation of the market. For example, if the same user is constantly targeted by the same ad, s/he becomes less likely to click. This means that the *slope* of the click rate function decreases as the number of impressions increases.

We assume that the number of clicks has the following shape (instead of being linear).
<img src="Plot 1.png" width="520">

We will approximate the function using a *piecewise linear* function.
<img src="Plot 3.png" width="520">

The good news is that we can model these functions using linear programs!

The modeling approach for this approximation is explained step-by-step below.

## Step 0: Slopes

The first step is to define the "slopes" on each piece of the function. This data is represented in the next dictionary:

In [53]:
pieces = ["pieceA","pieceB"]
slopesLBSlondon = {
    "pieceA": [0,5000,0.002],
    "pieceB": [5000,10000,0.0001],
}
# This means that between 0 and 5000 impressions, the clicks increase at a rate of 0.002
# Between 5000 and 10000 impressions, the clicks increase at a rate of 0.0001

## Step 1: Define the auxiliary variables

We define an auxiliary variable $y_{pieceA},y_{pieceB}$ for each piece. *Create an auxiliary variable for each segment of the function*.

In [ ]:
# Uncomment and complete the code below

auxiliary = {}
# for s in pieces:
#     ub = slopesLBSlondon[s][1]- slopesLBSlondon[s][0]
#     auxiliary[s] = 

### Q4.1: What should be the upper bound of the auxiliary variable?

*Insert your answer here:*



## Step 2: Add a coupling constraint

Now, we create a coupling constraint such that the sum of the auxiliary variables is equal to the corresponding allocation variable.

$$ y_{pieceA} + y_{pieceB} = x_{LBS,lon}$$

In other words, the allocation is "broken-down" into the two pieces.

In [ ]:
# Insert your code here


## Step 3: Modify the objective

In [ ]:
# We modify the objective function (we use the "slopes" instead of "CTR" for the allocation of "LBS" to"london")
m.setObjective(revenuePerKeyword['LBS','london']*quicksum(slopesLBSlondon[s][2]* auxiliary[s] for s in pieces)
              + quicksum(allocation[B,k]*revenuePerKeyword[B,k]*clickRatePerKeyword[k] for B in bSchools 
                         for k in keywords if B != "LBS" or k != 'london'), GRB.MAXIMIZE)

## Solve the new LP

In [ ]:
m.optimize()
printSolution()

### Q4.2. How was the optimal allocation affected (compared to Q2)? Why?

*Insert your answer here:*


### Q4.3. Suppose that the shape of the CTR function is now given below. Can we easily capture this in a linear program?

<img src="Plot 2.png" width="520">

*Insert your answer here:*


# Q5. More Practice

Using the same approach as Q3, you can now modify the click rates for all the keywords and business schools. Using the dictionary below, create a new model using the piecewise linear click rate functions specified by the dictionary `slopes`.

In [ ]:
pieces = ["pieceA","pieceB"]
slopes = {
    ('LBS','london','pieceA'): [0,5000,0.002],
    ('LBS','london','pieceB'): [5000,10000,0.0005],
    ('LBS','master_program', 'pieceA'): [0,1000,0.004], 
    ('LBS','master_program', 'pieceB'): [1000,3000,0.002], 
    ('LBS','data_science', 'pieceA'): [0,1000,0.004],
    ('LBS','data_science', 'pieceB'): [1000,3000,0.001],
    ('Imperial','london','pieceA'): [0,5000,0.002],
    ('Imperial','london','pieceB'): [5000,10000,0.0005],
    ('Imperial','master_program', 'pieceA'): [0,1000,0.004], 
    ('Imperial','master_program', 'pieceB'): [1000,3000,0.002], 
    ('Imperial','data_science', 'pieceA'): [0,1000,0.004],
    ('Imperial','data_science', 'pieceB'): [1000,3000,0.001]    
}

In [ ]:
# Write your code here

